In [1]:
import os

os.chdir("..")

In [2]:
import transformers
from transformers.utils import send_example_telemetry

send_example_telemetry("translation_notebook", framework="pytorch")

In [3]:
model_checkpoint = "csebuetnlp/banglat5_small"
# model_checkpoint = "t5-small"

In [4]:
split_config = {
    "train": "./datasets/correction_train.jsonl",
    "test": "./datasets/correction_test.jsonl",
    "val": "./datasets/correction_val.jsonl",
}

In [5]:
from datasets import load_dataset
import evaluate

raw_datasets = load_dataset("json", data_files=split_config)
metric = evaluate.load("sacrebleu")

In [6]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['from', 'to'],
        num_rows: 553064
    })
    test: Dataset({
        features: ['from', 'to'],
        num_rows: 69134
    })
    val: Dataset({
        features: ['from', 'to'],
        num_rows: 69133
    })
})

In [7]:
raw_datasets["train"][0]

{'from': 'কিন্তু ফের অসুস্থ বোধ করায় তাকে আবারও হাসপাতালে  ভর্তি  করা  হয় ।',
 'to': 'কিন্তু ফের অসুস্থ বোধ করায় তাকে আবারও হাসপাতালে ভর্তি করা হয়।'}

In [8]:
fake_preds = ["hello there", "general kenobi"]
fake_labels = [["hello there"], ["general kenobi"]]
metric.compute(predictions=fake_preds, references=fake_labels)

{'score': 0.0,
 'counts': [4, 2, 0, 0],
 'totals': [4, 2, 0, 0],
 'precisions': [100.0, 100.0, 0.0, 0.0],
 'bp': 1.0,
 'sys_len': 4,
 'ref_len': 4}

In [9]:
import sentencepiece

In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, legacy=True)
# tokenizer = AutoTokenizer.from_pretrained(
#     "./model_artifacts/pretrained_tokenizer_generic/"
# )

In [11]:
tokenizer("Hello, this one sentence!")

{'input_ids': [3948, 2845, 1214, 4, 4467, 6000, 29493, 47, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [12]:
tokenizer(["Hello, this one sentence!", "This is another sentence."])

{'input_ids': [[3948, 2845, 1214, 4, 4467, 6000, 29493, 47, 1], [11675, 1141, 25001, 29493, 26, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

In [13]:
tokenizer(["good morning ."])

{'input_ids': [[10593, 27552, 531, 1]], 'attention_mask': [[1, 1, 1, 1]]}

In [14]:
tokenizer(text_target="fa")

{'input_ids': [8077, 1], 'attention_mask': [1, 1]}

In [15]:
print(tokenizer(text_target=["Hello, this one sentence!", "This is another sentence."]))

{'input_ids': [[3948, 2845, 1214, 4, 4467, 6000, 29493, 47, 1], [11675, 1141, 25001, 29493, 26, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}


In [16]:
max_input_length = 128
max_target_length = 128
source_lang = "from"
target_lang = "to"


def preprocess_function(examples):
    inputs = examples["from"]
    targets = examples["to"]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    labels = tokenizer(
        text_target=targets, max_length=max_target_length, truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [17]:
preprocess_function(raw_datasets["train"][:2])

{'input_ids': [[36, 1892, 1941, 987, 2303, 169, 2027, 779, 802, 37, 21, 31, 1], [99, 20, 925, 7517, 17, 4, 873, 119, 1816, 14656, 7, 2820, 1562, 570, 882, 225, 14238, 1413, 141, 1618, 13, 412, 567, 16, 1911, 109, 710, 42, 19, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[36, 1892, 1941, 987, 2303, 169, 2027, 779, 802, 37, 21, 3, 1], [99, 1351, 17, 4, 119, 1816, 14656, 7, 2820, 186, 1413, 1525, 2528, 42, 19, 1]]}

In [18]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

In [19]:
from transformers import (
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
)

# model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint, vocab_size=30_000, ignore_mismatched_sizes=True)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [20]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
    evaluation_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=100,
    predict_with_generate=True,
    fp16=False,
    logging_steps=1,
    push_to_hub=False,
)

In [21]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [22]:
import numpy as np


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
    ]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [25]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"]
    .shuffle(42)
    .select(range(batch_size * 120)),
    eval_dataset=tokenized_datasets["train"].shuffle(42).select(range(batch_size * 10)),
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [26]:
tokenized_datasets["train"]

Dataset({
    features: ['from', 'to', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 553064
})

In [27]:
try:
    trainer.train()
except KeyboardInterrupt:
    print("wtf!")

wandb: Currently logged in as: mdmmn378. Use `wandb login --relogin` to force relogin


You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,2.570500,2.155434,30.862500,14.550000
2,2.200400,1.746299,36.069800,15.218800


wtf!


In [26]:
trainer.evaluate()

{'eval_loss': 0.44682106375694275,
 'eval_bleu': 67.185,
 'eval_gen_len': 13.3958,
 'eval_runtime': 0.3976,
 'eval_samples_per_second': 120.725,
 'eval_steps_per_second': 7.545,
 'epoch': 100.0}

# Test

In [26]:
import torch

In [27]:
torch.Tensor(tokenizer("আজ শুক্রবার").input_ids).int()

tensor([  92, 1251,    1], dtype=torch.int32)

In [41]:
input_ids = tokenizer(
    "কর্মসূচিটি সারােশের জেলা, হাগর ও পজেলাগুলিেও পালন করাবে।",
    return_tensors="pt",
).input_ids.cuda()

In [37]:
tokenized_datasets["test"].select(range(batch_size * 10))[:100]

{'from': ['তারা দমন করছেন ভিন্ন মতকে',
  'থারা উধাহরণ হিশেভে শফিখ রেহমান ও মাহফুঝ আনামের খথা উল্লেখ খরেছে।',
  'এর অযর্থ দাঁড়ায়, জাতী়য়ঔ নির্বাচনে হরেে গেলে নমাথয়া আঅকাশ ভেঙ্মগখে পষড়ার বিষয় থাকে।',
  'সোমবার দুপুরে ঠাকুরগাঁও সদর উপজেলা নারগুন এলাকরা বিএনপির নেতাকর্মীদের উদ্দ্যেশে বক্তব্য দেয়ার সময় এসব কথা বলেন তিনি',
  'বিনপি েয়রপরসন ালো জিয়র আর্থ্রাটিজের সমস্যাটা বে ে গেছে।',
  'একটি পরিবেশ থৈরি করত ে চান।',
  'ার্জিটি জগন্না পজো নির্াহী ফসারেরআ উকবর্যালয়ের অফিস আসকারী ফয়সাঝল আহমেদ গ্রহণ করউেছিলেফন।',
  'কর্মসূচিটি সারােশের জেলা, হাগর ও পজেলাগুলিেও পালন করাবে।',
  'সদ্য সমাপ্ত জাতীয় সংসদ নির্বাচনে তিনি আওয়ামী লীগের মনোনয়ন চেয়েছিলেন',
  'আনোয়ারা সণাত্তার নামে ঢ়72২ বঘছর বয়সী মহিলা এই আসনে প্রথশমবারের মতো ভোট দিয়েছেন।',
  'ইতভিমধ্যে শুক্রবার গণমথাধ্ঙযমে পাঠানঠোপ এক বিতবৃতিতে বিএনপির যুগ্ম মহাসচিবছ সালাহউদ্দিন আউহমেদ সরকএারের ঙবিরয়ুযদ্ধে অচিরেই আইন অমান্য ও অসহযোগ আন্দোলন শুরু করার ইঙ্গিত দিয়েছেন।ম',
  'কিন্তু প্রতিবারই তিনি ও তার দল ধরা খেয়ে যায়।',
  'পরে শপথ বাক্য পাঠ করানো হয়।',

In [38]:
input_ids

tensor([[   60,  1329, 28752,   647,  1465,    20,    10,   904,     3,     1]],
       device='cuda:0')

In [42]:
with torch.no_grad():
    ret = model.generate(input_ids=input_ids)
print(tokenizer.batch_decode(ret))

['<pad><extra_id_0> জেলা, হাগর ও জেলাগুলিতে পালন করা হবে।</s>']
